<a href="https://colab.research.google.com/github/AtacZy/Projects/blob/main/%D0%A0%D0%B0%D1%81%D1%87%D1%91%D1%82_%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Расчёт продуктовых метрик онлайн-платформы**


**1.   Retention** <br>
**2.   Rolling retention** <br>
**3.   Lifetime** <br>
**4.   Churn rate** <br>
**5.   Mau** <br>
**6.   Wau** <br>
**7.   Dau** <br>


---



## **Цель:**
Расчёт каждой продуктовой метрики из списка



---


## **Задачи:**
1. Посчитать каждую метрику
2. Дать небольшое заключение по каждой метрике
3. Сделать вывод о проделанной работе




---

## **Автор: AtacZy**



---

## **Выводы:**

**1.   Retention = 54.65116** <br>
**2.   Rolling retention = 29.06977** <br>
**3.   Lifetime = 14.804** <br>
**4.   Churn rate = 0.509** <br>
**5.   Mau = (133, 102.58333)** <br>
**6.   Wau = (84, 89.86792)** <br>
**7.   Dau = (47, 40.5589)** <br>

**Примечание:** Mau, Wau, Dau считаются двумя разными способами


###Данные для работы:
Файлы registration.csv и entries.csv

In [ ]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

###**Подготовка данных к работе**:

In [3]:
# Читаем каждый файл, разделяем по наименованию колонок и соединяем в один словарь
with open('entries.csv', 'r') as file:
  entries_row = file.readlines()[1:] # пропускаем заголовок

entries = [] # Собираем первый файл в список словарей
for row in entries_row:
  user_id = int(row.split(';')[0])
  entry_date = row.split(';')[1].strip()
  entries.append({'user_id':user_id, 'entry_date':entry_date})

with open('registrations.csv', 'r') as file:
  registration_row = file.readlines()[1:]

registrations = [] # Собираем второй файл в список словарей
for row in registration_row:
  user_id = int(row.split(';')[0])
  registration_date = row.split(';')[1].strip()
  registrations.append({'user_id':user_id, 'registration_date':registration_date})

In [4]:
# Убираем дубликаты в данных. Нужны только уникальные пользователи
unique_users = set()

filtred_entries = []

for entry in entries:
  user_id = entry['user_id']
  entry_date = entry['entry_date']

  if (user_id, entry_date) in unique_users:
    continue
  else:
    unique_users.add((user_id, entry_date))
    filtred_entries.append(entry)

###Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе

*   Сохраните результат в переменную retention_15_day
*   Примечание: результат округлите до 5 знаков после запятой



In [25]:
from datetime import datetime
from collections import defaultdict

# Фильтруем январские регистрации
january_registrations = [reg for reg in registrations if datetime.strptime(reg['registration_date'], '%Y-%m-%d').month == 1]

# Объединяем январские входы и дни регистрации по user_id
merged = []
for registration in january_registrations:
  for entry in filtred_entries:
    if entry['user_id'] == registration['user_id']:
      merged.append({"user_id": entry["user_id"], "registration_date": registration["registration_date"], "entry_date": entry["entry_date"]})

# Посчитаем количество дней с момента регистрации
days_after_registration = []
for m in merged:
  registration_date = datetime.strptime(m['registration_date'], '%Y-%m-%d')
  entry_date = datetime.strptime(m['entry_date'], '%Y-%m-%d')
  days = (entry_date - registration_date).days
  days_after_registration.append({"user_id": m["user_id"], "days_after_registration": days})

# Отфильтруем пользователей на 15 день
returned_15 = set()
for i in days_after_registration:
  if i['days_after_registration'] == 15:
    returned_15.add(i['user_id'])

# Считаем retention
retention_15_day = round(len(returned_15)/len(january_registrations) * 100, 5)
retention_15_day

54.65116

###Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты


*   Сохраните результат в перменную rolling_retention
*   Примечание: результат округлите до 5 знаков после запятой



In [27]:
# Создадим множество для пользоватетей 30 дня или после
rolling_retention_rates = set()

for i in days_after_registration:
  if i['days_after_registration'] >= 30:
    rolling_retention_rates.add(i['user_id'])

# Считаем rolling-retention
rolling_retention = round(len(rolling_retention_rates)/len(january_registrations) * 100, 5)
rolling_retention

29.06977

### Lifetime по всем пользователям, посчитанный как интеграл от n-day retention


*   Сохраните результат в переменную lifetime



In [19]:
from datetime import datetime

# Объедими датасет с регистрациями и уникальными входами пользователей
all_users_merged = []
for registration in registrations:
  for entry in filtred_entries:
    if entry['user_id'] == registration['user_id']:
      all_users_merged.append({"user_id": entry["user_id"], "registration_date": registration["registration_date"], "entry_date": entry["entry_date"]})

# Считаем кол-во дней с момента регистрации каждого пользователя
days_since_registration_all = []
for all in all_users_merged:
  registration_date = datetime.strptime(all['registration_date'], '%Y-%m-%d')
  entry_date = datetime.strptime(all['entry_date'], '%Y-%m-%d')
  days = (entry_date - registration_date).days
  days_since_registration_all.append({'user_id' : all['user_id'], 'days_since_registration_all' : days})

counts = {}
for entry in days_since_registration_all:
  days_since_registration = entry['days_since_registration_all']
  if days_since_registration not in counts:
    counts[days_since_registration] = 0
  counts[days_since_registration] += 1
each_day_ret = dict(sorted(counts.items()))

day_zero_count = each_day_ret[0]
lifetime = 0

for day, count in each_day_ret.items():
  lifetime += (count / day_zero_count)

lifetime = round(lifetime, 5)
lifetime

14.804

### Churn rate 29 дня, посчитанный по всем пользователям


*   Сохраните результат в переменную churn_29



In [23]:
chur_29 = set()

for i in days_since_registration_all:
  if i['days_since_registration_all'] >= 29:
    chur_29.add(i['user_id'])

churn_rate_29 = (len(registrations) - len(chur_29)) / len(registrations)
churn_rate_29

0.509

### Mau, wau, dau за последний месяц/неделю/день записей


*   Сохраните результат в переменные dec-mau, dec-wau, dec-dau соответсвенно
*   **Примечание**: последний месяц записей - декабрь. Поэтому mau рассчитываем для декабря (2021 года), для wau берем последнюю неделю - с 25 по 31 декабря, и для dau соответственно последний день - 31 декабря.



In [49]:
user_december_entry = []

for i in filtred_entries:
  if i['entry_date'][:4] == '2021' and i['entry_date'][5:7] == '12':
    user_december_entry.append(i)

# Считаем mau
mau_december = set()
for i in user_december_entry:
  mau_december.add(i['user_id'])

dec_mau = len(mau_december)
dec_mau

133

In [50]:
# Считаем wau

decs_wau = set()
for i in user_december_entry:
  if i['entry_date'][-2:] in ('25', '26', '27', '28', '29', '30', '31'):
    decs_wau.add(i['user_id'])

dec_wau = len(dec_wau)
dec_wau

84

In [51]:
# Считаем dau

decs_dau = set()
for i in user_december_entry:
  if i['entry_date'][-2:] == '31':
    dec_dau.add(i['user_id'])

dec_dau = len(dec_dau)
dec_dau

47

### Mau/wau/dau усредненные

*   Сохраните результат в переменные avg-mau, avg_wau, avg_dau соответственно



In [66]:
# Считаем MAU
from collections import defaultdict

# Создаём defaultdict для хранения уникальных юзеров каждого месяца
counts_avg = defaultdict(set)

# Пройдём по входам
# Выделим user_id и дату входа
# Вытащим год и месяц
# Используем год и месяц в качестве ключа

for line in entries_row:
  user_id, entry_date = line.strip().split(';')
  year, month = entry_date.split('-')[:2]
  key = f'{year}-{month}'
  counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_unique_users = {}
for key, value in counts_avg.items():
  count_unique_users[key] = len(value)

# Считаем усредненный mau
sum = 0
for key, value in count_unique_users.items():
  sum += value

avg_mau = round(sum / len(count_unique_users), 5)
avg_mau

102.58333

In [67]:
# Считаем WAU
from collections import defaultdict
from datetime import datetime

counts = defaultdict(set)

# Преобразуем строку с датой в объект datetime
# Получаем номер недели в году и год с помощью метода isocalendar()
# Формируем ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
for line in entries_row:
    user_id, entry_date = line.strip().split(';')
    date_obj = datetime.strptime(entry_date, '%Y-%m-%d')
    year, week_num, _ = date_obj.isocalendar()
    key = f"{year}-{week_num}"
    counts[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

# Считаем усредненный wau
sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_wau = round(sum/len(count_of_unique_user_ids), 5)
avg_wau

89.86792

In [68]:
# Считаем DAU
counts = defaultdict(set)

for line in entries_row:
    user_id, entry_date = line.strip().split(';')

    # В качестве ключа используем дату входа
    counts[entry_date].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_dau = round(sum/len(count_of_unique_user_ids), 5)
avg_dau

40.5589